In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import malaya

/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3361
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/ubuntu/.local/lib/python3.8/site-packages/malaya/tokenizer.py:202: FutureWarning: Possible nested set at position 3879
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
model = malaya.paraphrase.transformer(model = 'tiny-t5')

2022-11-24 14:58:18.666258: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-24 14:58:18.670233: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-24 14:58:18.670253: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-11-24 14:58:18.670256: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-11-24 14:58:18.670312: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.141.3
2022-11-24 14:58:18.670326: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] ke

In [4]:
from sacrebleu.metrics import BLEU, CHRF, TER

bleu = BLEU()
chrf = CHRF(word_order = 2)

In [6]:
import json

test = []
with open('shuffled-test.json') as fopen:
    for l in fopen:
        test.append(json.loads(l))
        
len(test)

5544

In [10]:
from tqdm import tqdm

batch_size = 1

results = []
for i in tqdm(range(0, len(test), batch_size)):
    input_ids = [s['translation']['src'] for s in test[i:i + batch_size]]
    outputs = model.greedy_decoder(input_ids)
    results.extend(outputs)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 5544/5544 [26:50<00:00,  3.44it/s]


In [11]:
filtered_left, filtered_right = [], []
for no, r in enumerate(results):
    if len(r):
        filtered_left.append(r)
        filtered_right.append(test[no]['translation']['tgt'])

In [12]:
refs = [filtered_right]
sys = filtered_left
r = bleu.corpus_score(sys, refs)
r.__dict__

{'name': 'BLEU',
 'score': 13.253918978157994,
 '_mean': -1.0,
 '_ci': -1.0,
 '_verbose': '38.6/16.3/9.4/5.8 (BP = 0.973 ratio = 0.974 hyp_len = 98419 ref_len = 101064)',
 'bp': 0.9734830243783323,
 'counts': [37961, 15099, 8219, 4762],
 'totals': [98419, 92875, 87331, 81787],
 'sys_len': 98419,
 'ref_len': 101064,
 'precisions': [38.57080441784615,
  16.257335127860028,
  9.411320149774994,
  5.822441219264675],
 'prec_str': '38.6/16.3/9.4/5.8',
 'ratio': 0.9738284651310061}